In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

# 1
В файле «baseball.csv» находится выборка с информацией по игрокам в бейсбол, включая
статистику их результативности, время участия в играх, лига, зарплата и т.д. Name (имя) нужно
считать идентификатором записи. Загрузите этот файл и произведите следующие действия для
кластерного анализа.

In [2]:
df = pd.read_csv("baseball.csv")
df = df.set_index("Name")

df_base = df.copy()

df.head()

# 2
Обработка пропусков. Переменная Salary (и log Salary) может содержать пропуски,
произведите подстановку пропусков методом согласно вашему варианту. Пересчитайте
logSalary как log(1+Salary), чтобы получить более симметричное распределение.

KnnImputer (neighbors=5)

In [3]:
knn_imp = KNNImputer(n_neighbors=5)

df[["Salary", "logSalary"]] = knn_imp.fit_transform(df[["Salary", "logSalary"]])

df["logSalary"] = np.log(1 + df["Salary"])

df_after_2 = df.copy()

df.head()

# 3
Нормализация переменных – приведите числовые переменные к близким шкалам с помощью
методов для вашего варианта и закодируйте категориальные с помощью OneHotEncoder.

MaxAbsScaler 

In [4]:
interval = [
    "CrAtBat", "CrBB", "CrHits", "CrHome", "CrRbi", "CrRuns", "logSalary", "nAssts", "nAtBat", "nBB", "nError",
    "nHits", "nHome", "nOuts", "nRBI", "nRuns", "Salary", "YrMajor"
]

nominal = [
    "Div", "Division", "League", "Position", "Team",
]

df[interval] = MaxAbsScaler().fit_transform(df[interval])

# df = pd.get_dummies(df, columns=nominal)

ohe = OneHotEncoder(sparse_output=False)
ohe.fit(df[nominal])

temp_df = pd.DataFrame(data=ohe.transform(df[nominal]), columns=ohe.get_feature_names_out(), index = df.index)
df.drop(columns=nominal, inplace=True)
df = pd.concat([df, temp_df], axis=1)

df.head()

# 4
С помощью восходящей иерархической кластеризации с выбранными параметрами
расстояния согласно вашему варианту постройте кластерную модель данных и дендрограмму
для топ 20 кластеров.

link=average, dist=euclidean

In [5]:
def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count
    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)
    dendrogram(linkage_matrix, **kwargs)
    plt.xlabel("Количество наблюдений в узле")


model = AgglomerativeClustering(linkage="average", metric="euclidean", n_clusters=20, compute_distances=True)
model = model.fit(df)
plot_dendrogram(model, truncate_mode="level", p=4)

# 5
Рассчитайте значение критерия pseudoF для вариантов кластеризации 2-20 кластеров,
постройте график зависимости критерия от числа кластеров и выберите оптимальное (первый
локальный пик критерия при обходе от малого числа кластеров к большому). Отметьте точку
на графике. Сколько кластеров получилось?

In [6]:
def sum_dist_to_center(X):
    center = np.mean(X, axis = 0)
    return ((X - center)**2).values.sum()


def choose_num_clusters(X, max_clust = 30):
    N = X.shape[0]
    Q = sum_dist_to_center(X)
    pseudo_f = np.array([])
    for G in range(2, max_clust + 1):
        clustering = KMeans(n_clusters = G, n_init="auto").fit(X)
        W = 0
        for l in range(G):
            elems = X[clustering.labels_ == l]
            W += sum_dist_to_center(elems)
        fisher_stat = ((Q - W)/(G - 1))/(W/(N - G))
        pseudo_f = np.append(pseudo_f, fisher_stat)
        
    plt.plot(range(2, max_clust + 1), pseudo_f)
    ind_best_clust = np.argmax(pseudo_f)
    plt.scatter(ind_best_clust + 2, pseudo_f[ind_best_clust], color="r", marker="D", s=50)
    plt.xlabel("Number of clusters")
    plt.ylabel("Pseudo-F")
    return ind_best_clust + 2


k = choose_num_clusters(df, max_clust=20)
k

# 6
С помощью метода проекции для вашего варианта постройте отображение на плоскость,
цветом точки укажите номер кластера.

PCA

In [7]:
pca = PCA(n_components=2)
features = pca.fit_transform(df)

plt.scatter(features[:, 0], features[:, 1], c=model.labels_)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# 7
Выполните кластеризацию сферическими кластерами с прототипом методом из вашего
варианта, также постройте проекцию как на шаге 6, определите наиболее типичного
представителя (по имени) в каждом из кластеров.

EM

In [8]:
gm = GaussianMixture(n_components=k, covariance_type="spherical")

clusters = gm.fit_predict(df)

plt.scatter(features[:, 0], features[:, 1], c=clusters)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [9]:
df7 = df.copy()
df7["cluster"] = clusters

# for i in range(k):
#     cur_cluster = df7[df7["cluster"] == i]
#     center = np.mean(cur_cluster, axis = 0)
#     print(np.sqrt(((cur_cluster - center) ** 2).sum(axis=1)).sort_values().index[0])

for i in range(k):
    cur_cluster = df7[df7["cluster"] == i]
    center = pd.Series(np.append(gm.means_[i], 0), index=df7.columns)
    print(np.sqrt(((cur_cluster - center) ** 2).sum(axis=1)).sort_values().index[0])

# 8
Реализуйте шаги 3-7 в виде функции или класса. 

In [10]:
def func8(df, interval, nominal):
    #plt.clf()
    # 3
    print("task 3")
    df[interval] = MaxAbsScaler().fit_transform(df[interval])

    # df = pd.get_dummies(df, columns=nominal)

    ohe = OneHotEncoder(sparse_output=False)
    ohe.fit(df[nominal])

    temp_df = pd.DataFrame(data=ohe.transform(df[nominal]), columns=ohe.get_feature_names_out(), index = df.index)
    df.drop(columns=nominal, inplace=True)
    df = pd.concat([df, temp_df], axis=1)
    
    # 4
    print("task 4")
    model = AgglomerativeClustering(linkage="average", metric="euclidean", n_clusters=20, compute_distances=True)
    model = model.fit(df)
    plot_dendrogram(model, truncate_mode="level", p=4)
    plt.show()
    
    # 5
    print("task 5")
    k = choose_num_clusters(df, max_clust=20)
    plt.show()
    print(k)
    
    # 6
    print("task 6")
    pca = PCA(n_components=2)
    features = pca.fit_transform(df)

    plt.scatter(features[:, 0], features[:, 1], c=model.labels_)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()
    
    # 7
    print("task 7")
    gm = GaussianMixture(n_components=k, covariance_type="spherical")

    clusters = gm.fit_predict(df)

    plt.scatter(features[:, 0], features[:, 1], c=clusters)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()
    
    df7 = df.copy()
    df7["cluster"] = clusters

    # for i in range(k):
    #     cur_cluster = df7[df7["cluster"] == i]
    #     center = np.mean(cur_cluster, axis = 0)
    #     print(np.sqrt(((cur_cluster - center) ** 2).sum(axis=1)).sort_values().index[0])
    
    for i in range(k):
        cur_cluster = df7[df7["cluster"] == i]
        center = pd.Series(np.append(gm.means_[i], 0), index=df7.columns)
        print(np.sqrt(((cur_cluster - center) ** 2).sum(axis=1)).sort_values().index[0])

In [11]:
func8(df_after_2, interval, nominal)